## Importing libraries for assistance in analysis

In [30]:
import pandas as pd
import numpy as np
import pickle
import requests

## Declaring constants used in the notebook

In [26]:
PRESENT_YEAR = 2017

## Reading the FB data stored in JSON format from the disk

In [51]:
def readData(filename):
    """
    Method reads the facebook Graph API dump data already stored on the
    disk in pickle format. The data is in JSON format
    :param filename : filename for the data stored on the disk
    :return : Json Data extracted from FB API that was stored on the disk
    """
    with open(filename, "rb") as fileHandler:
        data = pickle.load(fileHandler)    
    return data

filename = "../output/aashishFrnds"
data = readData(filename)

In [52]:
# data['friends']['data'] # a list of user's friends
data.keys()

dict_keys(['photos', 'work', 'likes', 'favorite_athletes', 'favorite_teams', 'birthday', 'tagged_places', 'sports', 'id', 'hometown', 'name', 'education', 'gender', 'family', 'location', 'friends', 'relationship_status'])

## Building User_Feature Matrix

In [53]:
def reqData(req):
    """
    Method requests Longitude and Latitude data from fb Graph API explorer.
    :return: data fetched from Graph API
    """
    accessToken = "EAACEdEose0cBANnzu3eimBYGfhWkhYZCSm8y9jTGHCW4ZA0bg8cFVsunX9pmXtVA2fzdLpVNqitcacIshNclvLxctIQw0N9Wtd5Fu3jF7DfCP4dxHAm5pknd0v1TpdG8dnCB60DmZBltRrIrL8KbmK7hkgB5OxUeoNF31Rq7TJyeheRqN3rtuCZAhJDf6orCVJEpY4dEJQZDZD"
    response = requests.get("https://graph.facebook.com/v2.10/" + req,
                 {'access_token':accessToken})
#     print (response)
    return str(response.json()["location"]["latitude"]), str(response.json()["location"]["longitude"])

def getFeatures(data):
    """
    Function to extract feautures like user_id, age, present_latitude, present_longitude from the given data
    :param data : JSON. Data extracted from FB graph API
    :return     : List of strings. e.g. [user_id, age, p_lat, p_log]
    """
    user_id = data['id']
    
    try:
        birthYear = data['birthday'][-4: ]
    except Exception as e:
        birthYear = None
    try:    
        age = str(PRESENT_YEAR - int(birthYear))
    except Exception as e:
        age = None
        
    try:        
        location_id = data['location']['id']
        query = location_id+"?fields=location"
        p_lat, p_long = reqData(query)
    except Exception as e:        
        p_lat, p_long = None, None
        
    return user_id, age, p_lat, p_long

def writeUserFeatureVector(fileHandler, *args):
    """
    Function write user feature vector for 1 user on to disk with the provided fileHandler
        e.g. 460881604288576 25 40.3 -70.456   (where 1st = ID, 2nd = Age, 3rd = Long, 4th = Lat)
    :param fileHandler : File Object.
    :param *args : List of arguments. Contains information like, age, location_lat, location_long
    :return : None
    """
    for item in args:
        fileHandler.write(str(item)+" ")
    fileHandler.write("\n")
    return None

def buildFeatureVectorForUsers(data):
    """
    Function to write feature vector of all users on disk
        e.g 460881604288576 25 40.3 -70.456 
            123145675555555 26 12.6  50.9
            897687345234534 32 34.5 -23.54
    :param data: JSON. Data retrived from FB graph API
    :return : None
    """
    visitedUser = set()
    with open("../data/userFeatureMatrix.txt", 'w') as fileHandler:   
        if data['id'] not in visitedUser:
            user_id, age, p_lat, p_long = getFeatures(data)
            writeUserFeatureVector(fileHandler, user_id, age, p_lat, p_long)
            visitedUser.add(user_id)
        
        for friend in data['friends']['data']:
            if friend['id'] not in visitedUser:
                user_id, age, p_lat, p_long = getFeatures(friend)
                writeUserFeatureVector(fileHandler, user_id, age, p_lat, p_long)
                visitedUser.add(user_id)
            try:
                for friend1 in friend['friends']['data']:
                    if friend1['id'] not in visitedUser['id']:
                        user_id, age, p_lat, p_long = getFeatures(friend)
                        writeUserFeatureVector(fileHandler, suser_id, age, p_lat, p_long)
                        visitedUser.add(user_id)
            except Exception as e:
                user_id, age, p_lat, p_long = friend1['id'], None, None, None
                if user_id not in visitedUser:
                    writeUserFeatureVector(fileHandler, user_id, age, p_lat, p_long)
        
        
    return None

buildFeatureVectorForUsers(data)

In [44]:
data['friends']['data'][0]['friends']['data'][0].keys()

dict_keys(['name', 'id'])